> Eğer `OPENAI_API_KEY` tanımlı değilse, terminale aşağıdakini yazın:  
> `export OPENAI_API_KEY="sk-..."`  
> ya da `.env` dosyasına bu satırı ekleyin ve Jupyter'ı yeniden başlatın.

In [55]:
# 00_settings_check    | OPENAI_API_KEY, pandas, openai sürüm çıktısı

# LIBS & BASIC SETUP
import os, json, time, sys, subprocess
from datetime import datetime

# 1 | Paketi mevcut değilse kur → sadece bir kez çalışır
for pkg in ("pandas", "openai"):
    try:
        __import__(pkg)
    except ImportError:
        subprocess.check_call([sys.executable, "-m", "pip", "install", pkg])

import pandas as pd
import openai

print("Pandas :", pd.__version__, "| OpenAI :", openai.__version__)



# 2 | API Key
openai.api_key = os.getenv("OPENAI_API_KEY")
if not openai.api_key:
    raise EnvironmentError("OPENAI_API_KEY bulunamadı → terminalde `export OPENAI_API_KEY=...`")

# 3 | Dosya kontrol
DATA_PATH = "cleaned_result.json"
if not os.path.exists(DATA_PATH):
    raise FileNotFoundError(f"'{DATA_PATH}' dosyası yok – çalışma dizini: {os.getcwd()}")

with open(DATA_PATH, encoding="utf-8") as f:
    raw = json.load(f)
df = pd.DataFrame(raw)
print("Veri yüklendi | Kayıt :", len(df))
df.head(2)


Pandas : 2.3.1 | OpenAI : 1.97.1
Veri yüklendi | Kayıt : 3000


,İş ID,Lokasyon,Sektör,İş Gönderim Saati,İade Talebi Saati,Hizmet Veren Geri Arama Saatleri,Hizmet Veren Arama Metinleri,Partner Geri Arama Saatleri,Partner Arama Metinleri,İade Durumu,İade Talebi Nedeni,Partner Red Sebebi,Müşteri Cevabı,Sistem Cevabı
0,115607,Şanlıurfa Haliliye,Buzdolabı Servisi,2025-07-22 16:00:02,2025-07-22 16:32:59,2025-07-22 16:30:46,Görüşmeleriniz kayıt altına alınmaktadır. Lütf...,None,None,Personel Kabul Etti,Servisnucreti kabul. Etmiyor iade istiyorum,None,None,None
1,115605,İstanbul Pendik,Bulaşık Makinesi Servisi,2025-07-22 15:58:56,2025-07-22 17:27:46,2025-07-22 17:26:54,Görüşmeleriniz kayıt altına alınmaktadır. Lütf...,None,None,Personel Kabul Etti,Müşteri dün kendisi halletmiş su kapalıymış su...,None,None,İş gönderimi ile müşterinin aranma saati arası...


In [56]:
# 01_data_cleaning    | JSON okuma, NaN düzeltme, datetime, duplicate

# CLEANING & FEATURES
date_cols = ["İş Gönderim Saati", "İade Talebi Saati"]
for c in date_cols:
    df[c] = pd.to_datetime(df[c], errors="coerce")

text_cols = [
    "İade Talebi Nedeni","Hizmet Veren Arama Metinleri","Partner Arama Metinleri",
    "Partner Red Sebebi","Müşteri Cevabı","Sistem Cevabı","Lokasyon","Sektör"
]
for c in text_cols:
    df[c] = df[c].fillna("").astype(str)

# Süre (dk)
df["Time_Elapsed_Minutes"] = (
    (df["İade Talebi Saati"] - df["İş Gönderim Saati"]).dt.total_seconds() / 60
)

# Hedef etiket → iki değer
df = df[df["İade Durumu"].isin(["Personel Kabul Etti","Personel Redetti"])].copy()
print("Kayıt (etiketli) :", len(df))


Kayıt (etiketli) : 2531


In [61]:
# 02_feature_engineering    | Time_Elapsed_Minutes

import json, pathlib
import textwrap

SYSTEM_PROMPT = (
    "You are an AI assistant representing the internal personnel of a service platform. "
    "Your role is to review refund requests submitted by service providers and make the final decision: "
    "'Accepted by Personnel' or 'Rejected by Personnel'. "
    "You must base your decision on the following structured information:\n\n"
    "- Reason for Refund Request: A short written justification from the provider.\n"
    "- Service Provider Call Transcripts: Transcript(s) of provider's calls with the customer.\n"
    "- Partner Call Transcripts: Transcript(s) of the partner's confirmation calls with the customer.\n"
    "- Time Elapsed Between Job Submission and Refund Request: Time in minutes.\n\n"
    "Analyze whether the provider gives a valid, timely, and reasonable explanation. "
    "Reject vague, insufficient, abusive claims, or cases where fault lies with the provider. "
    "Accept cases with clear external causes (e.g., customer unavailable, location inaccessible, technical issues beyond provider control). "
    "Do not assume anything beyond the given data.\n\n"
    "Respond ONLY with a JSON object:\n"
    "{'decision': 'Accepted by Personnel' or 'Rejected by Personnel', 'reason': 'short explanation in Turkish'}"
)

out = []
for _, row in df.iterrows():
    user_parts = [
        f"Reason for Refund Request: {row['İade Talebi Nedeni']}",
        f"Service Provider Call Transcripts: {row['Hizmet Veren Arama Metinleri']}",
        f"Partner Call Transcripts: {row['Partner Arama Metinleri']}",
        f"Time Elapsed Between Job Submission and Refund Request: "
        f"{'' if pd.isna(row['Time_Elapsed_Minutes']) else int(row['Time_Elapsed_Minutes'])} minutes."
    ]
    user_msg = "\n".join(p for p in user_parts if p.strip())

    decision = (
        "Accepted by Personnel"
        if row["İade Durumu"] == "Personel Kabul Etti"
        else "Rejected by Personnel"
    )

    # KISA ve ANLAMLI AÇIKLAMA
    raw_reason = row["İade Talebi Nedeni"] or "Sebep belirtilmemiş."
    short_reason = textwrap.shorten(raw_reason.replace("\n", " "), width=100, placeholder="...")

    assistant_msg = json.dumps(
        {"decision": decision, "reason": short_reason},
        ensure_ascii=False
    )

    out.append(json.dumps(
        {"messages": [
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": user_msg},
            {"role": "assistant", "content": assistant_msg}
        ]},
        ensure_ascii=False
    ))

JSONL_PATH = "fine_tuning_data.jsonl"
pathlib.Path(JSONL_PATH).write_text("\n".join(out), encoding="utf-8")
print("JSONL yazıldı →", JSONL_PATH, "| Satır:", len(out))


JSONL yazıldı → fine_tuning_data.jsonl | Satır: 2531


In [60]:
# 03_feature_engineering_extras   | arama sayıları → hizmet veren ve partner

def count_calls(value):
    if not value:
        return 0
    return len(str(value).split("||"))

df["HizmetVerenAramaSayisi"] = df["Hizmet Veren Arama Metinleri"].apply(count_calls)
df["PartnerAramaSayisi"] = df["Partner Arama Metinleri"].apply(count_calls)

print("Arama sayıları eklendi.")
df[["HizmetVerenAramaSayisi", "PartnerAramaSayisi"]].describe()


Arama sayıları eklendi.


,HizmetVerenAramaSayisi,PartnerAramaSayisi
count,2531.000000,2531.000000
mean,1.882655,0.835243
std,1.547936,1.298874
min,0.000000,0.000000
25%,1.000000,0.000000
50%,1.000000,0.000000
75%,2.000000,1.000000
max,24.000000,17.000000


In [51]:
# UPLOAD + FINE-TUNE
with open(JSONL_PATH, "rb") as f:
    file_id = openai.files.create(file=f, purpose="fine-tune").id
print("File ID:", file_id)

job = openai.fine_tuning.jobs.create(
    training_file=file_id,
    model="gpt-3.5-turbo-1106"
)
job_id = job.id
print("Job ID:", job_id)


File ID: file-Vkre3Ed7UeQpZTxQQvY1P4


BadRequestError: Error code: 400 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details.', 'type': 'invalid_request_error', 'param': None, 'code': 'exceeded_quota'}}

In [52]:
# FOLLOW JOB
import time, sys
while True:
    status = openai.fine_tuning.jobs.retrieve(job_id).status
    print(datetime.now().strftime("%H:%M:%S"), "→", status)
    if status in ("succeeded","failed"):
        break
    time.sleep(15)

if status == "succeeded":
    model_name = openai.fine_tuning.jobs.retrieve(job_id).fine_tuned_model
    print("Model hazır →", model_name)
else:
    print("Eğitim başarısız:", job_id)


NameError: name 'job_id' is not defined

In [53]:
# Bu hücre: Fine-tuning sonrası modelin çıktısını test etmek için kullanılır
# PDF'teki gereksinimi karşılar: "eğitilen modele prompt vererek sonuç al"

# 1. Değerlendirme fonksiyonu
def evaluate(model_name: str, reason: str, transcripts: str = "") -> str:
    qry = f"Reason for Refund Request: {reason}\nService Provider Call Transcripts: {transcripts}"
    resp = openai.chat.completions.create(
        model=model_name,
        messages=[
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": qry}
        ]
    )
    return resp.choices[0].message.content

# 2. Fine-tune eğitimi tamamlandıktan sonra model adını al:
# (Bunu bir defa çalıştırırsın, sonra final_model_name değişkeni oluşur.)
final_model_name = openai.fine_tuning.jobs.retrieve(job_id).fine_tuned_model
print("Model hazır:", final_model_name)

# 3. Test yapmak için aşağıdakini kullan (örnek):
test_reason = "Müşteri yanlış adres vermiş, hizmet iptal edilmiş."
print(evaluate(final_model_name, test_reason))


NameError: name 'job_id' is not defined